In [41]:
import pandas as pd
import os
import missingno as msno
from sklearn.impute import SimpleImputer

In [4]:
tracker = pd.read_csv('data/tracker.csv')

In [ ]:
# # combingin the
# directory = 'data/train_data/'

# # Initialize an empty dataframe
# combined_df = pd.DataFrame()

# # Loop through files in the directory
# for filename in os.listdir(directory):
#     if filename.endswith('.csv'):  # Assuming the files are in CSV format
#         file_path = os.path.join(directory, filename)
#         # Read the file into a dataframe
#         df = pd.read_csv(file_path)
#         # Add a column with the file name
#         df['File Name'] = filename
#         # Concatenate the dataframe to the combined dataframe
#         combined_df = pd.concat([combined_df, df], ignore_index=True)

# combined_df = combined_df.rename(columns={'Unnamed: 0': 'timestamp'})

In [35]:
combined_df = pd.read_csv('data/train_data/combined_data.csv')

In [13]:
msno.matrix(combined_df, figsize=(20, 10), fontsize=12)

(15,)

In [36]:
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], format='mixed')
combined_df['day'] = combined_df['timestamp'].dt.day


In [31]:
percentage = (nan_counts / len(combined_df)) * 100
#print(percentage)
#print(nan_counts)

In [38]:
# preparing for downsampling
combined_df.set_index('timestamp', inplace=True)

In [42]:
combined_df.fillna(method='ffill', inplace=True)


/tmp/ipykernel_158110/1234554365.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='ffill', inplace=True)
